## Descrição dos dados utilizados

<hr style="border: 1px solid #0984e3;">

Neste *notebook* foram utilizados dois conjuntos de dados para avaliar os algoritmos de classificação e agrupamento. A base de dados utilizada nos métodos supervisionados foi retirada do site [UCI](https://archive.ics.uci.edu/ml/datasets/Forest+type+mapping#), tais dados foram usados no artigo publicado por [Johnson et al., (2012)](https://www.tandfonline.com/doi/full/10.1080/01431161.2011.629637?scroll=top&needAccess=true), denominado "*Using geographically weighted variables for image classification*", no qual utilizou-se os índices espectrais para classificar dois tipos de árvores, *Cryptomeria japonica* (Sugi) e *Chamaecyparis obtusa* (Hinoki), e um tipo de floresta. A área de estudo apresentada na Figura 1 compreende a área florestal da prefeitura de Ibaraki do Japão. 

#### Área de estudo

<figure>
  <img src="https://raw.githubusercontent.com/dataAt/ml-aplicado-dados-espacial/master/src/img/study_area.png" alt="logo">
  <figcaption> Figura 1: Localização da área de estudo</figcaption>
</figure>

<br><br>
As imagens apresentadas acima foram imageadas pelo sensor ASTER a bordo do satélite Terra, nos períodos de Setembro de 2010, Março de 2011 e Maio de 2011, respectivamente. 

`Observação`: Os exemplos apresentados neste documento são feitos utilizando as bibliotecas [scikit-learn](https://scikit-learn.org/stable/), [keras](https://keras.io/) e [Somoclu](https://github.com/peterwittek/somoclu)

## Métricas de avaliação dos modelos

- Índice Kappa: Utilizado para medir o grau de concordância entre amostras.
- Score: Porcentagem de elementos classificados corretamente.

## Aprendizado supervisionado

Esta seção apresenta exemplos de técnicas de aprendizado de máquina supervisionados. Para os exemplos foi feita a utilização dos dados gerados no artigo [Using geographically weighted variables for image classification](https://www.tandfonline.com/doi/full/10.1080/01431161.2011.629637).

In [ ]:
# !conda install scikit-learn=0.22.1 -y

In [ ]:
# !conda install -c conda-forge somoclu -y

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# plt.style.use('ggplot')

def plot_dt(tree, feature_names, class_names):
    """Função criada para a visualização da árvore de decisão gerada 
    """
    from sklearn.tree import plot_tree
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    plot_tree(tree, ax = ax, feature_names=feature_names, class_names=class_names)

def plot_dendrogram(model, **kwargs):
    """Função para a geração de um dendograma de um modelo sklearn.cluster.AgglomerativeClustering
    
    See:
        Função retirada da documentação oficial do scikit-learn
    """
    from scipy.cluster.hierarchy import dendrogram
    
    plt.title('Agrupamento hierárquico')
    
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    plt.xlabel("Índice dos dados (Em parenteses representam a quantidade de elementos no grupo).")
    plt.show()
    

def plot_cm(cm_sklearn, labels):
    """Função para gerar matriz de confusão
    """
    
    import seaborn as sn
    
    df_cm = pd.DataFrame(cm_sklearn, index = [i for i in labels], columns = [i for i in labels])
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True)
    
# Configuração das classes
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
_ = le.fit(["d", "h", "o", "s"])

In [ ]:
# importando a função para calcular a matriz de confusão
from sklearn.metrics import confusion_matrix

In [ ]:
import numpy as np
import pandas as pd

Definindo as variáveis para o armazenamento dos dados de tipos de florestas.

In [ ]:
train_data = pd.read_csv('https://raw.githubusercontent.com/dataAt/ml-aplicado-dados-espacial/master/src/metodos-supervisionados/dados/forest_type/training.csv')
test_data = pd.read_csv('https://raw.githubusercontent.com/dataAt/ml-aplicado-dados-espacial/master/src/metodos-supervisionados/dados/forest_type/testing.csv')

O processo de treinamento de um algoritmo supervisionado, exige a existência de amostras já rotuladas sobre os dados, ou seja, para cada conjunto de entrada que será mapeado pelo algoritmo já deve existir uma resposta.

Além disto, para a validação da generalização do algoritmo, é necessário um conjunto de dados que ainda não tenha sido apresentado para o algoritmo, para que então, a generalização de seu conhecimento seja assegurada. Este conjunto de dados foi dividido em dados de `treino` e `teste` previamente, pelo criador dos dados.

Abaixo, o conjunto de dados carregado é separado em valores de `x`, que representam as entradas para o algoritmo, e valores `y`, que representam os valores desejados que são esperados do algoritmo retornar.

In [ ]:
data_columns = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

In [ ]:
x_train = train_data[data_columns]
y_train = train_data['class']

In [ ]:
x_test = test_data[data_columns]
y_test = test_data['class']

In [ ]:
## Definindo classes presentes no conjunto de dados
class_names = y_train.unique().tolist()
class_names

### K-Nearest Neighbors (KNN)
<hr style="border: 1px solid #0984e3;">

Esta seção apresenta a utilização do algoritmo `K-Nearest Neighbors (KNN)` através da biblioteca de *machine learning* scikit-learn

Para iniciar, todos os pacotes necessários para a utilização do algoritmo são importados.

In [ ]:
import numpy as np
import pandas as pd
from plotnine import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, cohen_kappa_score, accuracy_score

No `scikit-learn` o algoritmo KNN é oferecido através da classe `sklearn.neighbors.KNeighborsClassifier`. Para começar sua utilização, basta fazer a `instância` e treinar o modelo.

Faça inicialmente a instância da classe do KNN.

In [ ]:
knn = KNeighborsClassifier()

Com a instância feita, é necessário utilizar o método `fit` para que o algoritmo seja treinado.

In [ ]:
knn.fit(x_train, y_train)

Já temos o algoritmo knn treinado e pronto para realizar a classificação de outros dados. Para testar, utilizamos o método `predict` paras os dados de testes, que como citado, foram previamente separados.

In [ ]:
y_pred = knn.predict(x_test)

Utilizar as métricas para avaliar a qualidade dos resultados apresentados pelo algoritmo.

In [ ]:
# Calculando o índice Kappa
cohen_kappa_score(y_pred, y_test)

In [ ]:
# Calculando o Score
accuracy_score(y_pred, y_test)

Este algoritmo necessita da definição de um valor de `K`, mas a utilização acima não possui tal definição.

Caso o valor de `K` não seja especificado, utiliza-se o valor de `K` igual a 5. Porém, não há certeza de que este é o melhor valor de `K` para nosso conjunto de dados.

A definição deste valor pode não ser algo fácil, há bibliografias por exemplo, que recomendam que o valor de `K` sejam iguais a sqrt(n), onde n é a quantidade de amostras nos dados.

No nosso caso, vamos fazer vários testes para buscar o melhor valor de `K`. Cada teste consiste no treinamento do algoritmo com o valor de `K` diferente, sendo que este vária de 1 à 30.

In [ ]:
resultados = {'Kappa': [], 'k': [], 'Score': []}
for k_value in range(1, 15):
    resultados['k'].append(k_value)
    
    neigh = KNeighborsClassifier(n_neighbors=k_value)
    neigh.fit(x_train, y_train)
    
    y_pred = neigh.predict(x_test)
    
    resultados['Kappa'].append(cohen_kappa_score(y_pred, y_test))
    resultados['Score'].append(accuracy_score(y_pred, y_test))

resultados = pd.DataFrame(resultados)

# Visualizando os resultados
resultados.head()

In [ ]:
# Transformando os dados em long para facilitar o plot
resultados = resultados.melt('k', var_name = 'Medidas')

In [ ]:
(
    ggplot(resultados, aes(x = 'k', y = 'value', color = 'Medidas'))
        + geom_line()
        + theme_bw()
        + facet_grid('~Medidas', space = 'free_y', scales = 'free')
        + scale_x_continuous(breaks = np.arange(1, 17))
        + labs(
            title = 'Métricas de avaliação - KNN',
            x = 'Quantidade de vizinhos (K)',
            y = 'Acurácia'
        )
)

No plot gerado, foi possível perceber que a quantidade de elementos que nos traz os melhores resultados de predição é 2, desta forma podemos fazer a aplicação do método KNN utilizando `k` igual a 2.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)

In [ ]:
# Calculando o índice Kappa
cohen_kappa_score(y_pred, y_test)

In [ ]:
# Calculando o Score
accuracy_score(y_pred, y_test)

In [ ]:
# Resultados do KNN
y_pred = knn.predict(x_test)
knn_cm = confusion_matrix(y_test, y_pred)
plot_cm(knn_cm, "dhos")

### Árvore de decisão
<hr style="border: 1px solid #0984e3;">

Esta seção apresenta a utilização do algoritmo `Árvore de decisão` através da biblioteca de *machine learning* scikit-learn

Para iniciar, todos os pacotes necessários para a utilização do algoritmo são importados.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

No `scikit-learn` o algoritmo de árvore de decisão é oferecido através da classe `sklearn.tree.DecisionTreeClassifier`.
Basta definir `instância` e treinar o modelo.

In [ ]:
clf = DecisionTreeClassifier()

Para realizar o treinamento basta utilizamos o método `fit`, como feito no `KNN`.

In [ ]:
clf = clf.fit(x_train, y_train)

Com a árvore treinada, façamos a geração das métricas de avaliação do modelo.

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
# Calculando o índice Kappa
cohen_kappa_score(y_pred, y_test)

In [ ]:
# Calculando o Score
accuracy_score(y_test, y_pred)

Os resultados não foram ruins, o que é interessante.

In [ ]:
plot_dt(clf, data_columns, class_names)

**Problema**: a árvore gerada está muito complexa, tem muitas decisões sendo tomadas.

**Causa**: sinal de **overfitting**, onde a árvore de decisão não grava características gerais dos dados, mas sim, as características especificas dos dados.

**Solução**: realizar um `corte` na árvore e limitar sua profundidade a 2.

In [ ]:
# Com a árvore cortada, façamos todo o processo de avaliação e visualização de dados.
clf = DecisionTreeClassifier(max_depth=2)
clf = clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

In [ ]:
# Calculando o índice Kappa
cohen_kappa_score(y_pred, y_test)

In [ ]:
# Calculando o Score
accuracy_score(y_test, y_pred)

In [ ]:
plot_dt(clf, data_columns, class_names)

In [ ]:
# Resultados da árvore de decisão
y_pred = clf.predict(x_test)
clf_cm = confusion_matrix(y_test, y_pred)
plot_cm(clf_cm, "dhos")

### Redes Neurais Artificiais (RNA)
<hr style="border: 1px solid #0984e3;">

Esta seção apresenta a utilização de `Redes Neurais Artificiais` através da biblioteca de `Deep Learning` Keras.

Para iniciar, todos os pacotes necessários para a utilização do algoritmo são importados.

O modelo de rede neural criado possui 4 camadas, sendo elas:
   - 1° Camada: Entrada dos dados;
   - 2° Camada: Camada oculta;
   - 3° Camada: Camada oculta;
   - 4° Camada: Camada de saída (Classificação).
   
O modelo gerado é igual ao exibido na Figura abaixo, retiradas das [notas de aula - CS231n](http://cs231n.github.io/convolutional-networks/)

![](http://cs231n.github.io/assets/nn1/neural_net2.jpeg)

In [ ]:
from keras.layers import Dense
from keras.models import Sequential
from sklearn.preprocessing import OneHotEncoder, StandardScaler

Com os pacotes carregados, inicialmente faça a conversão dos dados categóricos que estão presentes no conjunto de dados. Tal conversão é necessária para que os modelos de rede neural gerados pelo `Keras` consigam entender que os dados possuem o tipo categórico.

In [ ]:
enc = OneHotEncoder()

y_train_factor = enc.fit_transform(train_data['class'].values[:, np.newaxis]).toarray()
y_test_factor = enc.fit_transform(test_data['class'].values[:, np.newaxis]).toarray()

Agora, vamos recuperar algumas informações do conjunto de dados, sendo elas, a quantidade de atributos e classes presentes nos dados.

In [ ]:
n_features = x_train.shape[1]
n_classes = y_train_factor.shape[1]

In [ ]:
model = Sequential(name='Modelo1')
model.add(Dense(4, input_dim=n_features, activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

Após a construção do modelo, vamos fazer sua `materialização` através do método `compile`.

In [ ]:
model.compile(loss='categorical_crossentropy', 
                      optimizer='adam', 
                      metrics=['accuracy'])

Vejamos o resumo do modelo que foi gerado

In [ ]:
model.summary()

Certo, o modelo está pronto para ser treinado, façamos isto com o método `fit`.

In [ ]:
history = model.fit(x_train, y_train_factor,
                         batch_size=5,
                         epochs=85,
                         verbose=1,
                         validation_data=(x_test, y_test_factor),)

Tendo o modelo treinado, façamos a avaliação do modelo com o método `evaluate`.

In [ ]:
model.evaluate(x_test, y_test_factor, verbose=0)

Agora, com o histórico do treinamento que foi gerado, vamos criar uma visualização dos resultados.

In [ ]:
history = pd.DataFrame(history.history)

In [ ]:
history.plot()

In [ ]:
# Resultados da RNA Número de épocas 85
y_pred = model.predict_classes(x_test)
model_cm = confusion_matrix(le.transform(y_test.map(lambda x: x.strip())), y_pred)
plot_cm(model_cm, "dhos")

In [ ]:
# Variação do número de épocas
model = Sequential(name='Modelo2')
model.add(Dense(4, input_dim=n_features, activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', 
                      optimizer='adam', 
                      metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train_factor,
                         batch_size=5,
                         epochs=48,
                         verbose=1,
                         validation_data=(x_test, y_test_factor),)

In [ ]:
model.evaluate(x_test, y_test_factor, verbose=0)

In [ ]:
history = pd.DataFrame(history.history)
history.plot()

In [ ]:
# Resultados da RNA Número de épocas 48
y_pred = model.predict_classes(x_test)
model_cm = confusion_matrix(le.transform(y_test.map(lambda x: x.strip())), y_pred)
plot_cm(model_cm, "dhos")

In [ ]:
# Variação do número de épocas
model = Sequential(name='Modelo3')
model.add(Dense(4, input_dim=n_features, activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', 
                      optimizer='adam', 
                      metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train_factor,
                         batch_size=5,
                         epochs=100,
                         verbose=1,
                         validation_data=(x_test, y_test_factor),)

In [ ]:
model.evaluate(x_test, y_test_factor, verbose=0)

In [ ]:
history = pd.DataFrame(history.history)
history.plot()

In [ ]:
# Resultados da RNA Número de épocas 100
y_pred = model.predict_classes(x_test)
model_cm = confusion_matrix(le.transform(y_test.map(lambda x: x.strip())), y_pred)
plot_cm(model_cm, "dhos")

In [ ]:
# Variação do número de camadas
model = Sequential(name='Modelo4')
model.add(Dense(4, input_dim=n_features, activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', 
                      optimizer='adam', 
                      metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train_factor,
                         batch_size=5,
                         epochs=48,
                         verbose=1,
                         validation_data=(x_test, y_test_factor),)

In [ ]:
model.evaluate(x_test, y_test_factor, verbose=0)

In [ ]:
history = pd.DataFrame(history.history)
history.plot()

In [ ]:
# Resultados da RNA Número camadas
# 4 Camadas ocultas com função ReLU e Softmax
y_pred = model.predict_classes(x_test)
model_cm = confusion_matrix(le.transform(y_test.map(lambda x: x.strip())), y_pred)
plot_cm(model_cm, "dhos")

In [ ]:
# Variação do número da camada final com função ReLU
model = Sequential(name='Modelo5')
model.add(Dense(4, input_dim=n_features, activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(n_classes, activation='relu'))

In [ ]:
model.compile(loss='categorical_crossentropy', 
                      optimizer='adam', 
                      metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train_factor,
                         batch_size=5,
                         epochs=48,
                         verbose=1,
                         validation_data=(x_test, y_test_factor),)

In [ ]:
model.evaluate(x_test, y_test_factor, verbose=0)

In [ ]:
history = pd.DataFrame(history.history)
history.plot()

In [ ]:
# Resultados da RNA Número camadas
# 4 Camadas ocultas com função ReLU e Softmax
y_pred = model.predict_classes(x_test)
model_cm = confusion_matrix(le.transform(y_test.map(lambda x: x.strip())), y_pred)
plot_cm(model_cm, "dhos")